In [2]:
from langchain_cohere import ChatCohere
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, XMLOutputParser
from langchain.schema.runnable import RunnableLambda, RunnableSequence, RunnableParallel

In [3]:
from pydantic_settings import BaseSettings

class Settings(BaseSettings):
    cohere_api_key: str
    
    class Config:
        env_file = ".env"
        extra = "ignore"
        
settings = Settings()

In [4]:
llm = ChatCohere(cohere_api_key = settings.cohere_api_key)
llm

ChatCohere(client=<cohere.client.Client object at 0x0000023D2B892F90>, async_client=<cohere.client.AsyncClient object at 0x0000023D2CEB7E00>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))

# Example 1

In [65]:
prompt_template = ChatPromptTemplate(
    [
        ("system", "You are a comedian. Talk about {topic}"),
        ("human", "Tell me {fact_count} of this topic"),
    ]
)
prompt_template

ChatPromptTemplate(input_variables=['fact_count', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='You are a comedian. Talk about {topic}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fact_count'], input_types={}, partial_variables={}, template='Tell me {fact_count} of this topic'), additional_kwargs={})])

In [66]:
chain = (
    prompt_template 
    | llm 
)

In [67]:
chain

ChatPromptTemplate(input_variables=['fact_count', 'topic'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='You are a comedian. Talk about {topic}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fact_count'], input_types={}, partial_variables={}, template='Tell me {fact_count} of this topic'), additional_kwargs={})])
| ChatCohere(client=<cohere.client.Client object at 0x00000250041F30E0>, async_client=<cohere.client.AsyncClient object at 0x00000250049CC050>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))

In [68]:
response = chain.invoke({"topic": "magic", "fact_count": 2 })
print(response.content)

Ah, magic! The art of making things disappear—like your wallet at a magician’s show. Here are two magical musings for you:

1. **The Magic of Misdirection**: Magicians are basically life coaches for your attention span. They say, "Look over here!" while your watch ends up in their pocket. It’s like a masterclass in "How to Distract Your Boss During a Meeting 101." Next time you’re late with a report, just wave a shiny object and whisper, "Ta-da!"  

2. **The Rabbit in the Hat**: Why is it always a rabbit? Did magicians just have a pet rabbit shortage in the 1800s? "Oh, I need a trick! Quick, grab Fluffy!" And now, centuries later, we’re still pulling bunnies out of hats like it’s the only animal that fits. What if they upgraded? "Ladies and gentlemen, behold… a fully assembled IKEA bookshelf!" Now *that’s* magic.  

Magic: where logic takes a vacation and rabbits work overtime.


In [70]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are know about the facts like animal. Talk about {animal}"),
        ("human", "Tell me {fact_count} of this topic"),
    ]
)
prompt_template

ChatPromptTemplate(input_variables=['animal', 'fact_count'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['animal'], input_types={}, partial_variables={}, template='You are know about the facts like animal. Talk about {animal}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fact_count'], input_types={}, partial_variables={}, template='Tell me {fact_count} of this topic'), additional_kwargs={})])

In [71]:
chain = (
    prompt_template 
    | llm 
    | StrOutputParser() 
)

In [72]:
chain

ChatPromptTemplate(input_variables=['animal', 'fact_count'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['animal'], input_types={}, partial_variables={}, template='You are know about the facts like animal. Talk about {animal}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fact_count'], input_types={}, partial_variables={}, template='Tell me {fact_count} of this topic'), additional_kwargs={})])
| ChatCohere(client=<cohere.client.Client object at 0x00000250041F30E0>, async_client=<cohere.client.AsyncClient object at 0x00000250049CC050>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))
| StrOutputParser()

In [73]:
type(chain)

langchain_core.runnables.base.RunnableSequence

In [74]:
print(chain.invoke({"animal": "cat",  "fact_count": 3, 'name':"riya"}))

Sure! Here are three interesting facts about cats:

1. **Cats have an incredible sense of balance**: Thanks to their flexible spine and powerful tail, cats can easily navigate narrow spaces and land on their feet after a fall. This ability is known as the "righting reflex," which allows them to twist their bodies mid-air to land feet-first.

2. **Cats can make over 100 different vocal sounds**: While dogs typically have about 10 vocalizations, cats are much more verbose. They use a variety of meows, purrs, chirps, and hisses to communicate with humans and other cats. Each cat’s meow is unique and can convey different emotions or needs.

3. **Cats spend 70% of their lives sleeping**: On average, cats sleep for 12–16 hours a day, with some sleeping up to 20 hours. This behavior stems from their ancestral hunting instincts, as they conserve energy for short bursts of activity. Even domestic cats retain this trait, though they don’t need to hunt for food!

Let me know if you'd like to lear

In [25]:
parser = StrOutputParser()
parser

StrOutputParser()

In [27]:
from langchain_core.messages import AIMessage

llm_response = AIMessage(content = "this is ai message")
llm_response

AIMessage(content='this is ai message', additional_kwargs={}, response_metadata={})

In [28]:
parser.invoke(llm_response)

'this is ai message'

In [57]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are helpful assistant, going to tell about a country"
            "Your response should be in the format of below json"
            "Format -> country_name, national_language, currency, capital, these should be the keys and the answer should be the values."
            "Do not add any extra answers and dont include any markdown codes. be sure the response properly terminated."
        ),
        ("human", "Tell me about {country}") 
    ]
)

In [42]:
prompt_template.invoke({"country": "China"})

ChatPromptValue(messages=[SystemMessage(content='You are helpful assistant, going to tell about a countryYour response should be in the format of below jsonFormat -> country_name, national_language, currency, capital, these should be the keys and the answer should be the values.Do not add any extra answers and dont include any markdown codes. be sure the response properly terminated.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about China', additional_kwargs={}, response_metadata={})])

In [51]:
chain = prompt_template | llm | JsonOutputParser()
chain

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are helpful assistant, going to tell about a countryYour response should be in the format of below jsonFormat -> country_name, national_language, currency, capital, these should be the keys and the answer should be the values.Do not add any extra answers and dont include any markdown codes. be sure the response properly terminated.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='Tell me about {country}'), additional_kwargs={})])
| ChatCohere(client=<cohere.client.Client object at 0x000001F7EA7B6210>, async_client=<cohere.client.AsyncClient object at 0x000001F7EA7B7390>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))
| JsonOutputParser()

In [52]:
response = chain.invoke({"country": "United States"})

In [55]:
response

{'country_name': 'United States of America',
 'national_language': 'English',
 'currency': 'United States Dollar (USD)',
 'capital': 'Washington, D.C.'}

In [58]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are helpful assistant, going to tell about a country"
            "Your response should be in the format of below format **XML**"
            "Format -> country_name, national_language, currency, capital, these should be the tags and the answer should be the values."
            "Do not add any extra answers and dont include any markdown codes. be sure the response properly terminated."
        ),
        ("human", "Tell me about {country}") 
    ]
)

In [61]:
chain = prompt_template | llm | XMLOutputParser()
chain

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are helpful assistant, going to tell about a countryYour response should be in the format of below format **XML**Format -> country_name, national_language, currency, capital, these should be the tags and the answer should be the values.Do not add any extra answers and dont include any markdown codes. be sure the response properly terminated.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='Tell me about {country}'), additional_kwargs={})])
| ChatCohere(client=<cohere.client.Client object at 0x000001F7EA7B6210>, async_client=<cohere.client.AsyncClient object at 0x000001F7EA7B7390>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))
| XMLOutputParser()

In [62]:
response = chain.invoke({"country": "India"})

In [64]:
print(response)

{'country': [{'country_name': 'India'}, {'national_language': 'Hindi, English'}, {'currency': 'Indian Rupee (INR)'}, {'capital': 'New Delhi'}]}


# Chain Inner Working

### Example 2 Without pipeline

In [75]:
# Task 1
prompt_template = ChatPromptTemplate.from_messages(          
    [
        ("system", "You are know about the facts like animal. Talk about {animal}"),
        ("human", "Tell me {fact_count} facts"),
    ]
)
prompt_template

ChatPromptTemplate(input_variables=['animal', 'fact_count'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['animal'], input_types={}, partial_variables={}, template='You are know about the facts like animal. Talk about {animal}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['fact_count'], input_types={}, partial_variables={}, template='Tell me {fact_count} facts'), additional_kwargs={})])

In [78]:
# individual runnable (steps in the chain)
format_prompt = RunnableLambda(lambda x: prompt_template.format_prompt(**x))
invoke_model = RunnableLambda(lambda x: llm.invoke(x.to_messages()))
parse_output = RunnableLambda(lambda x: x.content)


In [81]:
# create runnablesequence
chain = RunnableSequence(first = format_prompt, middle = [invoke_model], last = parse_output)

In [84]:
response = chain.invoke({"animal": "Tiger","fact_count": 3})
print(response)

Here are three fascinating facts about tigers:

1. **Tigers are the largest cat species**: Among all wild cats, tigers are the biggest, with males (called "tigers") weighing up to 660 pounds (300 kg) and measuring up to 10 feet (3 meters) in length, including their tail.

2. **Each tiger has unique stripes**: Just like human fingerprints, no two tigers have the same stripe pattern. Their stripes act as camouflage in their natural habitat, helping them blend into the tall grass and ambush prey.

3. **Tigers are excellent swimmers**: Unlike most cats, tigers are strong and confident swimmers. They often swim between islands or across rivers to hunt, cool off, or mark their territory. Some tigers have been observed swimming distances of up to 18 miles (29 kilometers) in a single day!


# Types of chain

## Sequence Chain

In [75]:
animal_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You like telling facts and you tell facts about {animal}."),
        ("human", "Tell me {count} facts.")
    ]
)
animal_prompt_template

ChatPromptTemplate(input_variables=['animal', 'count'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['animal'], input_types={}, partial_variables={}, template='You like telling facts and you tell facts about {animal}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['count'], input_types={}, partial_variables={}, template='Tell me {count} facts.'), additional_kwargs={})])

In [76]:
translation_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a translator and convert the provided text into {language}."),
        ("human", "Translate the following text to {language}: {text}"),
    ]
)
translation_template

ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, partial_variables={}, template='You are a translator and convert the provided text into {language}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, template='Translate the following text to {language}: {text}'), additional_kwargs={})])

In [82]:
count_words = RunnableLambda(lambda x: f"Word count: {len(x.split())}\n{x}")
prepare_for_translation = RunnableLambda(lambda output: {"text": output, "language":"Saurashtra"})


In [83]:
chain = (
    animal_prompt_template 
    | llm 
    | StrOutputParser() 
    | prepare_for_translation 
    | translation_template 
    | llm 
    | StrOutputParser()
    | count_words
    )

In [84]:
chain

ChatPromptTemplate(input_variables=['animal', 'count'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['animal'], input_types={}, partial_variables={}, template='You like telling facts and you tell facts about {animal}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['count'], input_types={}, partial_variables={}, template='Tell me {count} facts.'), additional_kwargs={})])
| ChatCohere(client=<cohere.client.Client object at 0x00000250041F30E0>, async_client=<cohere.client.AsyncClient object at 0x00000250049CC050>, model='command-a-03-2025', cohere_api_key=SecretStr('**********'))
| StrOutputParser()
| RunnableLambda(lambda output: {'text': output, 'language': 'Saurashtra'})
| ChatPromptTemplate(input_variables=['language', 'text'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language'], input_types={}, pa

In [85]:
result = chain.invoke({"animal": "Elephant", "count": 2})

In [86]:
print(result)

Word count: 105
અહીં હાથીઓ વિશે બે રોચક તાણાં છે:

1. **હાથીઓને કોઈપણ જમીની જાનવરની સૌથી લાંબી ગર્ભાવસ્થા છે.** એક માદા હાથી (ગાય) તેની બાછળને લગભગ **22 મહિના** (લગભગ 2 વર્ષ) સુધી પોતાની કોખ માં રાખે છે અને જન્મ આપે છે. આ વિસ્તારિત ગર્ભાવસ્થા બાછળને પૂરી રીતે વિકાસ કરવા અને જન્મના સમયે સાપેક્ષ સ્વતંત્ર બનવાની મજબૂતી આપે છે.

2. **હાથીઓ લો-ફ્રીક્વેન્સી આવાજ વાળા અન્ફ્રાસાઉન્ડ વાપરીને લાંબા દૂરીઓ પર સંચાર કરી શકે છે.** આ આવાજો, જે માનવ સાંભળવાની સીમા કરતાં નીચે છે, જમીન અને હવા દ્વારા વધુ કિલોમીટર સુધી પ્રવાહિત થઈ શકે છે. હાથીઓ તેમના જૂથો સાથે સમન્વય કરવા, ખતરીની ચેતવણી આપવા અને જોડાઓ શોધવા માટે અન્ફ્રાસાઉન્ડ વાપરે છે.


In [52]:
a = "hi"

# Parallel Chaining

In [5]:
summery_template = ChatPromptTemplate.from_messages(
    [
        ("system", " You are a movie critic"),
        ("human", " Provide a brief summary of the movie {movie_name}"),
    ]
)

In [6]:
# Define Plot Analysis step

def analyze_plot(plot):
    plot_template = ChatPromptTemplate.from_messages(
        [
            ("system", "you are a movie Critic."),
            ("human", "Analyze the plot: {plot}. What are its strengths and weaknesses?"),
        ]
    )
    return plot_template.format_prompt(plot=plot)

In [7]:
# Define characters analyses step

def analyze_characters(characters):
    character_template = ChatPromptTemplate.from_messages(
        [
            ("system", "you are a movie Critic."),
            ("human", "Analysze the character: {characters} What are its strengths and weaknesses?")

        ]
    )
    return character_template.format_prompt(characters = characters)

In [8]:
# combine analyses into a final verdict

def combine_verdicts(plot_analysis, character_analysis):
    return f"Plot Analysis:\n {plot_analysis} \n\n Character Analysis: \n{character_analysis}"

In [9]:
# Simplify branches with LCEL

plot_branch_chain = (
    RunnableLambda(lambda x: analyze_plot(x)) | llm | StrOutputParser()
)

In [10]:
character_branch_chain = (
    RunnableLambda(lambda x: analyze_characters(x)) | llm | StrOutputParser()
)

In [11]:
chain = (
    summery_template
    | llm
    | StrOutputParser()
    | RunnableParallel(branches = {"plot": plot_branch_chain, "characters": character_branch_chain})
    | RunnableLambda(lambda x: combine_verdicts(x["branches"]["plot"],x["branches"]["characters"]))
)

In [12]:
response = chain.invoke({"movie_name": "Master Tamil Movie"})

In [13]:
print(response)

Plot Analysis:
 **Strengths of *Master***:

1. **Star Power and Performances**:  
   The film leverages the charisma and screen presence of its lead actors, Vijay and Vijay Sethupathi. Vijay delivers a balanced performance as John Durairaj, effortlessly switching between mass hero moments and emotional vulnerability. Vijay Sethupathi, as Bhavani, is menacing and compelling, creating a formidable antagonist that elevates the conflict. Their face-off is one of the film's highlights, drawing audiences in with their chemistry and intensity.

2. **High-Octane Action Sequences**:  
   Lokesh Kanagaraj’s direction shines in the action set pieces, which are choreographed with precision and scale. The film’s mass moments, particularly those featuring Vijay, are designed to cater to his fan base while also appealing to general audiences. The climax confrontation between John and Bhavani is particularly gripping, leaving a lasting impact.

3. **Emotional Depth and Social Commentary**:  
   *Maste